# Clustering with K-Means

In [1]:
# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import pickle

## Pre-processing for K-Means
Add to dataframe: 
- number of complaints per company
- average word count of complaints, per company

#### Add number of complaints per company

In [3]:
df_complaint_num = pd.read_pickle('df_complaint_num.pkl')
df_complaint_num.head()

,num_complaints,cum_sum,cum_perc
PORTFOLIO RECOVERY ASSOCIATES INC,900,900,4.12
ENCORE CAPITAL GROUP INC.,694,1594,7.30
Resurgent Capital Services L.P.,512,2106,9.65
CAPITAL ONE FINANCIAL CORPORATION,473,2579,11.81
ERC,418,2997,13.73


In [4]:
def num_complaints_per_company(df_complaint_num):
    df_complaint_num = df_complaint_num.rename_axis('Company').reset_index()
    df_complaint_num.sort_values(by=['Company'], inplace=True)
    complaints_per_company = df_complaint_num[['Company', 'num_complaints']]
    return complaints_per_company

In [6]:
complaints_per_company = num_complaints_per_company(df_complaint_num)
complaints_per_company

,Company,num_complaints
587,1st Franklin Financial Corporation,5
349,2288984 Ontario Inc.,9
1230,"3rd Generation, Inc.",1
1648,"4M Collections, LLC",1
1640,"A & A North American Financial, LLC",1
...,...,...
1085,"ZenResolve, LLC",2
1375,eCon Credit LP,1
827,"eMoneyUSA Holdings, LLC",3
805,"iQuantified Management Services, LLC",3


In [8]:
company_top_prob = pd.read_pickle('company_top_prob.pkl')
company_top_prob

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
Company,,,,,,,,,,,,,,,,,,,,
"ACIMA CREDIT, LLC",0.000001,0.357143,0.428195,0.142858,0.356915,0.214286,0.277875,0.071429,0.000005,0.000002,0.531724,0.000001,1.000000e-06,0.290881,0.286787,0.439723,1.000000e-06,0.148562,0.344567,0.000969
AES/PHEAA,0.000001,0.464286,0.281508,0.053572,0.250094,0.214286,0.214982,0.250000,0.482576,0.000001,0.297766,0.144534,1.000000e-06,0.299323,0.141373,0.295638,7.143445e-02,0.200685,0.149157,0.215823
AFNI INC.,0.082570,0.137615,0.168184,0.238533,0.196649,0.376147,0.230811,0.155964,0.009246,0.018955,0.189477,0.110126,8.256964e-02,0.426620,0.216091,0.222741,7.339535e-02,0.277587,0.303640,0.373855
ALLY FINANCIAL INC.,0.111112,0.244445,0.148646,0.066668,0.133041,0.222223,0.422170,0.044445,0.047494,0.025315,0.266179,0.044627,1.000000e-06,0.207192,0.088685,0.201502,1.000000e-06,0.113988,0.128400,0.151483
"AMCOL Systems, Inc.",0.060607,0.090910,0.157752,0.060607,0.093054,0.272728,0.084689,0.151516,0.000003,0.497695,0.160451,0.030774,9.090991e-02,0.280312,0.106938,0.266755,6.060694e-02,0.277147,0.089983,0.284617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Westlake Services, LLC",0.148149,0.185186,0.163579,0.074075,0.074151,0.222223,0.116777,0.222223,0.051184,0.000073,0.226124,0.074100,7.407493e-02,0.151396,0.111823,0.222889,7.407493e-02,0.125704,0.219354,0.170282
"Williams & Fudge, Inc",0.000002,0.375000,0.291950,0.041668,0.385793,0.083334,0.594284,0.041668,0.251496,0.000001,0.512023,0.125421,1.000000e-06,0.501086,0.151818,0.245006,4.159179e-02,0.049262,0.120947,0.755132
"Windham Professionals, Inc.",0.000001,0.076924,0.155120,0.153847,0.347486,0.000001,0.803863,0.000001,0.000003,0.076924,0.242175,0.230768,1.000000e-06,0.294979,0.230770,0.185806,1.000000e-06,0.079564,0.104992,0.466119


In [9]:
# add to company_top_prob dataframe a column for number of complaints per Company
company_top_prob_2 = pd.merge(company_top_prob, complaints_per_company, left_index=True, right_on='Company')
company_top_prob_2 = company_top_prob_2.set_index('Company')
company_top_prob_2

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,num_complaints
Company,,,,,,,,,,,,,,,,,,,,,
"ACIMA CREDIT, LLC",0.000001,0.357143,0.428195,0.142858,0.356915,0.214286,0.277875,0.071429,0.000005,0.000002,...,0.000001,1.000000e-06,0.290881,0.286787,0.439723,1.000000e-06,0.148562,0.344567,0.000969,14
AES/PHEAA,0.000001,0.464286,0.281508,0.053572,0.250094,0.214286,0.214982,0.250000,0.482576,0.000001,...,0.144534,1.000000e-06,0.299323,0.141373,0.295638,7.143445e-02,0.200685,0.149157,0.215823,56
AFNI INC.,0.082570,0.137615,0.168184,0.238533,0.196649,0.376147,0.230811,0.155964,0.009246,0.018955,...,0.110126,8.256964e-02,0.426620,0.216091,0.222741,7.339535e-02,0.277587,0.303640,0.373855,109
ALLY FINANCIAL INC.,0.111112,0.244445,0.148646,0.066668,0.133041,0.222223,0.422170,0.044445,0.047494,0.025315,...,0.044627,1.000000e-06,0.207192,0.088685,0.201502,1.000000e-06,0.113988,0.128400,0.151483,45
"AMCOL Systems, Inc.",0.060607,0.090910,0.157752,0.060607,0.093054,0.272728,0.084689,0.151516,0.000003,0.497695,...,0.030774,9.090991e-02,0.280312,0.106938,0.266755,6.060694e-02,0.277147,0.089983,0.284617,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Westlake Services, LLC",0.148149,0.185186,0.163579,0.074075,0.074151,0.222223,0.116777,0.222223,0.051184,0.000073,...,0.074100,7.407493e-02,0.151396,0.111823,0.222889,7.407493e-02,0.125704,0.219354,0.170282,27
"Williams & Fudge, Inc",0.000002,0.375000,0.291950,0.041668,0.385793,0.083334,0.594284,0.041668,0.251496,0.000001,...,0.125421,1.000000e-06,0.501086,0.151818,0.245006,4.159179e-02,0.049262,0.120947,0.755132,24
"Windham Professionals, Inc.",0.000001,0.076924,0.155120,0.153847,0.347486,0.000001,0.803863,0.000001,0.000003,0.076924,...,0.230768,1.000000e-06,0.294979,0.230770,0.185806,1.000000e-06,0.079564,0.104992,0.466119,13


#### Add average word count of complaints, per company

In [17]:
df_split = pd.read_pickle('df_split.pkl')
df_split

,orig_index,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,0,2020-05-07,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,I received an alert from XXXX XXXX on XX/XX/XX...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",LA,700XX,None,Consent provided,Web,05/07/20,Closed with explanation,Yes,NaN,3642003
1,2,2020-02-11,Debt collection,Other debt,Took or threatened to take negative or legal a...,Seized or attempted to seize your property,See uploaded document dated XXXX ; CHASE BANK ...,None,JPMORGAN CHASE & CO.,GA,301XX,None,Consent provided,Web,02/11/20,Closed with explanation,Yes,NaN,3527527
2,3,2020-05-27,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,On XX/XX/XXXX I pulled my credit report and sa...,Company believes it acted appropriately as aut...,Monterey Financial Services LLC,NY,None,None,Consent provided,Web,05/27/20,Closed with explanation,Yes,NaN,3670227
3,4,2020-01-11,Debt collection,Other debt,Written notification about debt,Didn't receive notice of right to dispute,My XXXX ( XXXX ) Account number for Internet s...,None,AFNI INC.,FL,320XX,None,Consent provided,Web,01/11/20,Closed with explanation,Yes,NaN,3492864
4,5,2020-01-22,Debt collection,Medical debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,AMCOL Systems ( a debt collector ) reported a ...,None,"AMCOL Systems, Inc.",TX,775XX,None,Consent provided,Web,01/22/20,Closed with explanation,Yes,NaN,3504908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18151,21825,2020-05-23,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,"I was made aware, that once again, XXXX XXXX c...",Company has responded to the consumer and the ...,The Receivable Management Services LLC,WV,258XX,None,Consent provided,Web,05/23/20,Closed with explanation,Yes,NaN,3666313
18152,21827,2020-05-03,Debt collection,Credit card debt,Attempts to collect debt not owed,Debt was paid,using Lowe 's Advantage card I made online fro...,Company has responded to the consumer and the ...,SYNCHRONY FINANCIAL,WA,981XX,None,Consent provided,Web,05/03/20,Closed with monetary relief,Yes,NaN,3635213
18153,21828,2020-04-05,Debt collection,Credit card debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,In XX/XX/XXXX I paid the full amount of my XXX...,None,Alliance Data Card Services,TX,None,None,Consent provided,Web,04/05/20,Closed with non-monetary relief,Yes,NaN,3593729
18154,21829,2020-03-28,Debt collection,Credit card debt,Took or threatened to take negative or legal a...,Threatened to sue you for very old debt,XX/XX/XXXX my sister called to inform me that ...,Company has responded to the consumer and the ...,WELLS FARGO & COMPANY,CA,934XX,None,Consent provided,Web,04/01/20,Closed with explanation,Yes,NaN,3584339


In [18]:
# count number of words in each document/narrative, put into a dictionary 
def word_count_each_doc(df_split):
    word_cnt_dict = {}
    for index, complaint in enumerate(df_split['Consumer complaint narrative']):
        word_cnt = len(str(complaint).split())
        word_cnt_dict[index] = word_cnt
    df_word_cnt = pd.DataFrame(word_cnt_dict.values(), columns = ['word_cnt'])
    return df_word_cnt

In [19]:
df_word_cnt = word_count_each_doc(df_split)
df_word_cnt

,word_cnt
0,130
1,118
2,339
3,423
4,98
...,...
18151,89
18152,314
18153,1087
18154,200


In [20]:
# add column for word count of each document: merge 'df_word_cnt' and df_split
df_split_2 = pd.merge(df_split, df_word_cnt, left_index=True, right_index=True)
df_split_2.head()

,orig_index,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,word_cnt
0,0,2020-05-07,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,I received an alert from XXXX XXXX on XX/XX/XX...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",LA,700XX,None,Consent provided,Web,05/07/20,Closed with explanation,Yes,NaN,3642003,130
1,2,2020-02-11,Debt collection,Other debt,Took or threatened to take negative or legal a...,Seized or attempted to seize your property,See uploaded document dated XXXX ; CHASE BANK ...,None,JPMORGAN CHASE & CO.,GA,301XX,None,Consent provided,Web,02/11/20,Closed with explanation,Yes,NaN,3527527,118
2,3,2020-05-27,Debt collection,Other debt,Written notification about debt,Didn't receive enough information to verify debt,On XX/XX/XXXX I pulled my credit report and sa...,Company believes it acted appropriately as aut...,Monterey Financial Services LLC,NY,None,None,Consent provided,Web,05/27/20,Closed with explanation,Yes,NaN,3670227,339
3,4,2020-01-11,Debt collection,Other debt,Written notification about debt,Didn't receive notice of right to dispute,My XXXX ( XXXX ) Account number for Internet s...,None,AFNI INC.,FL,320XX,None,Consent provided,Web,01/11/20,Closed with explanation,Yes,NaN,3492864,423
4,5,2020-01-22,Debt collection,Medical debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,AMCOL Systems ( a debt collector ) reported a ...,None,"AMCOL Systems, Inc.",TX,775XX,None,Consent provided,Web,01/22/20,Closed with explanation,Yes,NaN,3504908,98


In [21]:
# added column: was 19, now 20 columns
df_split_2.shape

(18156, 20)

In [25]:
# get average number of words in a document, per Company
def avg_word_count_per_company(df_split_2):
    # group dataframe by Company, get average word count of complaints on per Company basis, as integers
    avg_len_complaint_by_company = df_split_2.groupby('Company')['word_cnt'].agg(np.mean).astype(int)
    
    # put into dataframe
    avg_len_complaint_by_company = pd.DataFrame(avg_len_complaint_by_company)
    
    # rename column for average word count
    avg_len_complaint_by_company.rename(columns={'word_cnt':'avg_word_cnt'}, inplace=True)
    
    #reset index
    avg_len_complaint_by_company = avg_len_complaint_by_company.reset_index()
    
    return avg_len_complaint_by_company

In [26]:
avg_len_complaint_by_company = avg_word_count_per_company(df_split_2)
avg_len_complaint_by_company

,Company,avg_word_cnt
0,"ACIMA CREDIT, LLC",234
1,AES/PHEAA,175
2,AFNI INC.,156
3,ALLY FINANCIAL INC.,117
4,"AMCOL Systems, Inc.",116
...,...,...
332,"Westlake Services, LLC",133
333,"Williams & Fudge, Inc",298
334,"Windham Professionals, Inc.",162
335,World Acceptance Corporation,93


In [27]:
# add to company_top_prob_2 dataframe a column for average length of complaints by Company
company_top_prob_2 = pd.merge(company_top_prob_2, avg_len_complaint_by_company, left_index=True, right_on='Company')
company_top_prob_2 = company_top_prob_2.set_index('Company')
company_top_prob_2

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,num_complaints,avg_word_cnt
Company,,,,,,,,,,,,,,,,,,,,,
"ACIMA CREDIT, LLC",0.000001,0.357143,0.428195,0.142858,0.356915,0.214286,0.277875,0.071429,0.000005,0.000002,...,1.000000e-06,0.290881,0.286787,0.439723,1.000000e-06,0.148562,0.344567,0.000969,14,234
AES/PHEAA,0.000001,0.464286,0.281508,0.053572,0.250094,0.214286,0.214982,0.250000,0.482576,0.000001,...,1.000000e-06,0.299323,0.141373,0.295638,7.143445e-02,0.200685,0.149157,0.215823,56,175
AFNI INC.,0.082570,0.137615,0.168184,0.238533,0.196649,0.376147,0.230811,0.155964,0.009246,0.018955,...,8.256964e-02,0.426620,0.216091,0.222741,7.339535e-02,0.277587,0.303640,0.373855,109,156
ALLY FINANCIAL INC.,0.111112,0.244445,0.148646,0.066668,0.133041,0.222223,0.422170,0.044445,0.047494,0.025315,...,1.000000e-06,0.207192,0.088685,0.201502,1.000000e-06,0.113988,0.128400,0.151483,45,117
"AMCOL Systems, Inc.",0.060607,0.090910,0.157752,0.060607,0.093054,0.272728,0.084689,0.151516,0.000003,0.497695,...,9.090991e-02,0.280312,0.106938,0.266755,6.060694e-02,0.277147,0.089983,0.284617,33,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Westlake Services, LLC",0.148149,0.185186,0.163579,0.074075,0.074151,0.222223,0.116777,0.222223,0.051184,0.000073,...,7.407493e-02,0.151396,0.111823,0.222889,7.407493e-02,0.125704,0.219354,0.170282,27,133
"Williams & Fudge, Inc",0.000002,0.375000,0.291950,0.041668,0.385793,0.083334,0.594284,0.041668,0.251496,0.000001,...,1.000000e-06,0.501086,0.151818,0.245006,4.159179e-02,0.049262,0.120947,0.755132,24,298
"Windham Professionals, Inc.",0.000001,0.076924,0.155120,0.153847,0.347486,0.000001,0.803863,0.000001,0.000003,0.076924,...,1.000000e-06,0.294979,0.230770,0.185806,1.000000e-06,0.079564,0.104992,0.466119,13,162


In [28]:
# pickle company_top_prob_2
# company_top_prob_2.to_pickle('company_top_prob_2.pkl')

## Rename topic numbers